In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
import os
import gc

# 1. 환경 설정
DATA_PATH = "../../data/raw/"
OOF_PATH = "./oof_data/"
DEVICE = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

if not os.path.exists(OOF_PATH):
    os.makedirs(OOF_PATH)

def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)

# 2. 데이터 로드 및 전처리 (기존 로직 유지)
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
train_data = train_data.drop(train_data[train_data.familysize > 50].index)
train_y = (2 - train_data['voted'].to_numpy()).astype(np.float32)

drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE', 'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE', 'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
             'index', 'hand']
replace_dict = {'education': str, 'engnat': str, 'married': str, 'urban': str}
train_x = train_data.drop(drop_list + ['voted'], axis=1).astype(replace_dict)
train_x = pd.get_dummies(train_x)
train_x_t = torch.tensor(train_x.to_numpy().astype(np.float32), dtype=torch.float32)
train_y_t = torch.tensor(train_y, dtype=torch.float32)

# 스케일링
train_x_t_std = train_x_t.clone()
train_x_t_std[:, :20] = (train_x_t_std[:, :20] - 3.) / 2.
train_x_t_std[:, 20] = (train_x_t_std[:, 20] - 5.) / 4.
train_x_t_std[:, 21:31] = (train_x_t_std[:, 21:31] - 3.5) / 3.5

# 3. 모델 정의 (m1, m2, m4 공통)
class MLP(nn.Module):
    def __init__(self, input_dim=91, hidden_dim=180):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.BatchNorm1d(hidden_dim), nn.LeakyReLU(),
            nn.Linear(hidden_dim, 32), nn.BatchNorm1d(32), nn.LeakyReLU(), nn.Linear(32, 1)
        )
    def forward(self, x): return self.net(x)

class SNNModel(nn.Module):
    def __init__(self, input_dim=91):
        super(SNNModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256), nn.SELU(), nn.AlphaDropout(0.1),
            nn.Linear(256, 128), nn.SELU(), nn.AlphaDropout(0.05),
            nn.Linear(128, 64), nn.SELU(), nn.Linear(64, 1)
        )
        for m in self.modules():
            if isinstance(m, nn.Linear): nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='linear')
    def forward(self, x): return self.net(x)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.3, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha, self.gamma = alpha, gamma
    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        return torch.mean(self.alpha * (1 - pt)**self.gamma * BCE_loss)

# 4. OOF 생성 및 저장 함수 (REPEAT 반영)
def run_oof_process(model_type, model_prefix, n_repeat, n_skfold, n_epoch, batch_size):
    print(f"\n>>> Running OOF Process: {model_prefix} (Repeat:{n_repeat}, Fold:{n_skfold}, Epoch:{n_epoch}, BS:{batch_size})")
    
    # 여러 번의 Repeat 결과를 평균내기 위한 공간
    total_oof_preds = np.zeros(len(train_x_t))
    
    overall_pbar = tqdm(range(n_repeat), desc=f"{model_prefix} Total Repeats")
    
    for r in overall_pbar:
        skf = StratifiedKFold(n_splits=n_skfold, random_state=r+42, shuffle=True)
        repeat_oof_preds = np.zeros(len(train_x_t))
        
        fold_pbar = tqdm(enumerate(skf.split(train_x_t, train_y_t)), total=n_skfold, desc=f"R{r+1} Folds", leave=False)
        for f, (t_idx, v_idx) in fold_pbar:
            t_loader = DataLoader(
                TensorDataset(train_x_t_std[t_idx], train_y_t[t_idx]), 
                batch_size=batch_size, 
                shuffle=True,
                drop_last=True
            )
            v_loader = DataLoader(
                TensorDataset(train_x_t_std[v_idx], train_y_t[v_idx]), 
                batch_size=batch_size, 
                shuffle=False
            )
            
            if model_type == 'm1': model = MLP(91, 180).to(DEVICE); criterion = nn.BCEWithLogitsLoss()
            elif model_type == 'm2': model = MLP(91, 180).to(DEVICE); criterion = nn.BCEWithLogitsLoss()
            elif model_type == 'm4': model = SNNModel(91).to(DEVICE); criterion = FocalLoss()
                
            optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
            best_val_auc = 0
            best_v_preds = None
            
            for epoch in range(n_epoch):
                model.train()
                for xx, yy in t_loader:
                    optimizer.zero_grad()
                    loss = criterion(model(xx.to(DEVICE)).view(-1), yy.to(DEVICE))
                    loss.backward(); optimizer.step()
                
                model.eval()
                v_fold_preds = []
                with torch.no_grad():
                    for vx, _ in v_loader:
                        v_fold_preds.extend(torch.sigmoid(model(vx.to(DEVICE)).view(-1)).cpu().numpy())
                
                auc = roc_auc_score(train_y_t[v_idx].numpy(), v_fold_preds)
                if auc > best_val_auc:
                    best_val_auc = auc
                    best_v_preds = v_fold_preds
                
                fold_pbar.set_postfix({'AUC': f'{auc:.5f}', 'Best': f'{best_val_auc:.5f}'})
            
            repeat_oof_preds[v_idx] = best_v_preds
            del model; gc.collect()
            
        total_oof_preds += repeat_oof_preds / n_repeat
        overall_pbar.set_postfix({'Running_AUC': f'{roc_auc_score(train_y, total_oof_preds / (r+1/n_repeat)):.5f}'})

    # 최종 OOF AUC 계산 및 저장
    total_auc = roc_auc_score(train_y, total_oof_preds)
    save_name = f"{OOF_PATH}{model_prefix}_AUC_{total_auc:.5f}.npy"
    np.save(save_name, total_oof_preds)
    print(f"\n[DONE] {model_prefix} | OOF AUC: {total_auc:.5f} | Saved: {save_name}")

# 5. 모델별 조건 맞춰 실행
# m1: SOTA 모델 조건 (공식기록 기반)
run_oof_process('m1', 'exp11_sota', n_repeat=5, n_skfold=7, n_epoch=48, batch_size=72)

# m2: AUC Optimized 모델 조건
run_oof_process('m2', 'exp12_auc', n_repeat=5, n_skfold=7, n_epoch=48, batch_size=72)

# m4: SNN-Focal (Exp 15 조건)
run_oof_process('m4', 'exp15_snn', n_repeat=5, n_skfold=7, n_epoch=60, batch_size=128)


>>> Running OOF Process: exp11_sota (Repeat:5, Fold:7, Epoch:48, BS:72)


exp11_sota Total Repeats:   0%|          | 0/5 [00:00<?, ?it/s]

R1 Folds:   0%|          | 0/7 [00:00<?, ?it/s]

R2 Folds:   0%|          | 0/7 [00:00<?, ?it/s]

R3 Folds:   0%|          | 0/7 [00:00<?, ?it/s]

...

R2 Folds:   0%|          | 0/7 [00:00<?, ?it/s]

R3 Folds:   0%|          | 0/7 [00:00<?, ?it/s]

R4 Folds:   0%|          | 0/7 [00:00<?, ?it/s]

R5 Folds:   0%|          | 0/7 [00:00<?, ?it/s]


[DONE] exp15_snn | OOF AUC: 0.77330 | Saved: ./oof_data/exp15_snn_AUC_0.77330.npy
